In [1]:
%pylab inline
from scipy.interpolate import interp1d
from read_LyaRT import wls
from os import listdir
import os.path

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Important numbers

v_th=12.4
c=300000.0 #km/s
LyA_freq=2.47*(10.0**15) #Hz (frequency at rest)

In [3]:
#Paths
galaxy_path = "./data/galaxy/"
outflow_path = "./data/outflow/"
images_path = "./data/galaxyoutflowimages/"
data_path = "./data/"

#Galaxy Filenames
galaxy_filenames = array(sorted(listdir(galaxy_path)))
number_galaxies = len(galaxy_filenames)
#galaxy_velocities = [60, 125] #(km/s)

#Outflow Filenames
outflow_filenames = array(sorted(listdir(outflow_path)))
number_outflows = len(outflow_filenames)
#outflow_velocities = [100.0, 200.0] #(km/s)
#outflow_lognHs = [20.0, 20.0625, 20.125, 20.1875, 20.25, 20.3125, 20.375, 20.4375, 20.5, 20.5625, 20.625, 20.6875,
#                  20.75, 20.8125, 20.875, 20.9375, 21.0, 21.0625, 21.125, 21.1875, 21.25, 21.3125, 21.375, 21.4375,
#                  21.5, 21.5625, 21.625, 21.6875, 21.75, 21.8125, 21.875, 21.9375, 22.0, 22.0625, 22.125, 22.1875,
#                  22.25, 22.3125, 22.375, 22.4375, 22.5]

In [7]:
filename = "galaxyoutflow"
spectrum_file = open(data_path+filename+".csv",'w')
spectrum_file.write("v_gal,v_out,lognh,x,y\n")

In [ ]:
#The first for is to try with the number of galaxies (different velocities)
for i in range(number_galaxies):
    
    #Get file with galaxy spectrum and its content
    gal_filename = galaxy_filenames[i]
    gal_velocity = int((gal_filename.split("v")[1]).split("a")[0])
    gal_logtau = int((gal_filename.split("t")[1]).split("v")[0])
    gal_spectrum = loadtxt(galaxy_path+gal_filename)
    gal_velocities = gal_spectrum[:,0]
    gal_intensities = gal_spectrum[:,1]
    gal_wavelengths = c*10**13/(LyA_freq*(1-gal_velocities/c)) # angstroms (*10**3 km_m *10**10 m_ang)
    
    #As the values inside the galaxy file can have a smaller interval in wavelength than the outflow, they have
    #to be extended in order to be equal, this process is started here and completed in the next for cycle.
    gal_number_wavelengths = len(gal_wavelengths)
    gal_delta_wavelength = gal_wavelengths[2]-gal_wavelengths[1]
    gal_initial_wavelength = gal_wavelengths[0]
    gal_final_wavelength = gal_wavelengths[gal_number_wavelengths-1]
    
    for j in range(number_outflows):
        
        #Get file with outflow
        out_filename = outflow_filenames[j]
        out_lognH = float((out_filename.split("_")[2]).split("nH")[1])
        out_velocity = float((out_filename.split("_")[3]).split("vmax")[1])
        
        wl0, wl = wls(outflow_path+out_filename)  
        h=histogram(wl0, bins=299)
        out_intensities_flat, out_wavelenghts_flat = h[0], h[1]
        out_intensity_flat_max = amax(out_intensities_flat)
        
        #Set the maximum intensity of galaxy to the same one of the outflow 
        
        gal_intensity_max = amax(gal_intensities)
        gal_intensities = gal_intensities*out_intensity_flat_max/gal_intensity_max

        #The extension of galaxy wavelengths is continued here
        out_initial_wavelenght_flat = amin(out_wavelenghts_flat)
        out_final_wavelenght_flat = amax(out_wavelenghts_flat)
        number_wavelengths_below = int(ceil((gal_initial_wavelength - out_initial_wavelenght_flat)/gal_delta_wavelength))
        number_wavelengths_above = int(ceil((out_final_wavelenght_flat - gal_final_wavelength)/gal_delta_wavelength))
        
        intensities_below = zeros(number_wavelengths_below)
        intensities_above = zeros(number_wavelengths_above)
        wavelengths_below = zeros(number_wavelengths_below)
        wavelengths_above = zeros(number_wavelengths_above)
        
        gal_initial_wavelength_copy = gal_initial_wavelength
        for k in range(number_wavelengths_below):
            gal_initial_wavelength_copy -= gal_delta_wavelength
            wavelengths_below[number_wavelengths_below-k-1] = gal_initial_wavelength_copy
    
        gal_final_wavelength_copy = gal_final_wavelength
        for k in range(number_wavelengths_above):
            gal_final_wavelength_copy += gal_delta_wavelength
            wavelengths_above[k] = gal_final_wavelength_copy
        
        gal_extended_intensities = concatenate((intensities_below, gal_intensities, intensities_above))
        gal_extended_wavelenghts = concatenate((wavelengths_below, gal_wavelengths, wavelengths_above))
        
        #Now with the wavelengths and intensities of the galaxy extended we can make the interpolation function        
        
        #Interpolation function
        f_interpolation = interp1d(gal_extended_wavelenghts, gal_extended_intensities)  
        
        #Evaluate wl0 in the interp. function
        intensities = f_interpolation(wl0)
        
        #Get final spectrum
        fig=figure()

        image_filename = "tgal2E"+str(gal_logtau)+"vgal"+str(gal_velocity)+"lognH"+str(out_lognH)+"vout"+str(out_velocity)
    
        spectrum = hist(wl, bins=100, weights=intensities, histtype='step')
        
        y=spectrum[0]
        x_wavelenght=spectrum[1]
        x=c*(1-(c*10**13)/(x_wavelenght*LyA_freq)) #in velocity units

        for i in range (len(y)):
            spectrum_file.write(str(gal_velocity)+","+str(out_velocity)+","+str(out_lognH)+","+str(x[i])+","+str(y[i])+"\n")

        xlabel("$\lambda$ (Angstrom)")
        ylabel("Intensity (Arbitrary Units)")
        legend(["$v_{gal}$ = "+str(gal_velocity)+" km/s" + "\n $lognH$ = "+str(out_lognH)+"\n $v_{out}$ = "+str(out_velocity)+" km/s"])
        savefig(images_path+image_filename+".png",format = 'png')
        close()  
        
spectrum_file.close()